In [1]:
import gymnasium as gym
import torch

In [8]:
env = gym.make("CartPole-v0")
n_state = env.observation_space.shape[0] 
n_action = env.action_space.n
n_episode = 100

/Users/arghasreebanerjee/opt/anaconda3/envs/XAI_week1/lib/python3.10/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


In [4]:
def run_episode(env, weight): 
    state = env.reset()[0]  # initial state
    total_reward = 0
    is_done = False
    while not is_done:
        state = torch.from_numpy(state).float()
        multiply = torch.matmul(state, weight)
        # print(f'State ({state.size()}) X Weight ({weight.size()}) = {multiply.size()}')
        # print('Choosing action based on the weights of invidual actions ... ', multiply,'\n')
        action = torch.argmax(multiply) # previous line 
        state, reward, is_done, _, info = env.step(action.item()) # Instead of sampling the action from a uniform distribution directly, we assign weights and then sample in previous line
        total_reward += reward
    return total_reward
        

In [14]:
best_total_reward = 0
best_weight = torch.rand(n_state, n_action)
noise_scale = 0.01 # Hill- climbing works by taking increasing the weight by some random noise, if the rewards increase in 1 episode, change the weights, or keep it the same. 
# the noise_scale is a scaling factor.

In [30]:
total_rewards=[]
for episode in range(n_episode):
    weight = best_weight+ torch.rand(n_state, n_action)
    reward = run_episode(env, weight)
    if reward > best_total_reward:
        best_total_reward = reward
        best_weight = weight
    total_rewards.append(reward)
    print(f'Episode {episode+1}: Rewards {reward}')

Episode 1: Rewards 275.0
Episode 2: Rewards 2941.0
Episode 3: Rewards 419.0
Episode 4: Rewards 1697.0
Episode 5: Rewards 2008.0
Episode 6: Rewards 872.0
Episode 7: Rewards 703.0
Episode 8: Rewards 405.0
Episode 9: Rewards 5826.0


KeyboardInterrupt: 

In [25]:
print(f'Average rewards = {sum(total_rewards)/n_episode}')

Average rewards = 112.81


If in the next episode, the rewards have increased, then it is a good idea to walk away from the current weight values. So decrease the scalling factor. \
If not, then don't add so much noise.

Things to worry about:
- How much should the noise increase or decrease? 
- Set a lower/upper bound so that the noise scale is not too low. 


In [34]:
best_total_reward = 0
best_weight = torch.rand(n_state, n_action)
noise_scale = 0.01 
total_rewards=[]
n_episode = 200

for episode in range(n_episode):
    weight = best_weight + torch.rand(n_state, n_action)
    reward = run_episode(env, weight)
    if reward >= best_total_reward:
        best_total_reward = reward
        best_weight = weight
        noise_scale = max(noise_scale/2, 1e-4)
    else:
        noise_scale = min(noise_scale*2, 2)
    total_rewards.append(reward)
    print(f'Episode {episode+1}: Rewards {reward}')

Episode 1: Rewards 9.0
Episode 2: Rewards 26.0
Episode 3: Rewards 10.0
Episode 4: Rewards 24.0
Episode 5: Rewards 17.0
Episode 6: Rewards 45.0
Episode 7: Rewards 57.0
Episode 8: Rewards 46.0
Episode 9: Rewards 44.0
Episode 10: Rewards 78.0
Episode 11: Rewards 57.0
Episode 12: Rewards 69.0
Episode 13: Rewards 103.0
Episode 14: Rewards 111.0
Episode 15: Rewards 513.0
Episode 16: Rewards 46.0
Episode 17: Rewards 41.0
Episode 18: Rewards 110.0
Episode 19: Rewards 46.0
Episode 20: Rewards 36.0
Episode 21: Rewards 120.0
Episode 22: Rewards 163.0
Episode 23: Rewards 122.0
Episode 24: Rewards 59.0
Episode 25: Rewards 36.0
Episode 26: Rewards 89.0
Episode 27: Rewards 167.0
Episode 28: Rewards 42.0
Episode 29: Rewards 55.0
Episode 30: Rewards 170.0
Episode 31: Rewards 200.0
Episode 32: Rewards 96.0
Episode 33: Rewards 55.0
Episode 34: Rewards 84.0
Episode 35: Rewards 121.0
Episode 36: Rewards 63.0
Episode 37: Rewards 242.0
Episode 38: Rewards 56.0
Episode 39: Rewards 264.0
Episode 40: Rewards 44

KeyboardInterrupt: 

In [35]:
print(f'Average rewards = {sum(total_rewards)/n_episode}')

Average rewards = 37.195
